In [2]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from tec_an import TecAn
import numpy as np
from data_util import *
import threading
import multiprocessing
from data_agent import DataAgent, TacProcess
from sklearn.model_selection import train_test_split

In [3]:
def save(data_set, prefix = "", base_dir = "data/"):
    trainX = data_set[0]
    trainY = data_set[1]
    valX = data_set[2]
    valY = data_set[3]
    final_path = path + base_dir
    train_path = "{}{}trainX.npy".format(final_path, prefix)
    np.save(train_path, trainX)
    np.save("{}{}trainY.npy".format(final_path, prefix), trainY)
    np.save("{}{}valX.npy".format(final_path, prefix), valX)
    np.save("{}{}valY.npy".format(final_path, prefix), valY)
    print("Saving {} with {}".format(train_path, trainX.shape))

def get_y_data(ohlc, shift = -1):
    combined_data = ohlc
    #combined_data['return'] = np.log(combined_data / combined_data.shift(1))
    returns = (ohlc / ohlc.shift(shift))
    combined_data['return'] = returns
    combined_data['direction'] = np.where(combined_data['return'] < 1, 1, 0)
    #print(combined_data)
    #combined_data.dropna(inplace=True)
    return combined_data['direction']

def split(x, y, split, shuffle=False):
    trainX, valX, trainY, valY = train_test_split(np.array(x), np.array(y), test_size=split, shuffle=shuffle)
    print("Completed: {} {} {} {}".format(trainX.shape, trainY.shape, valX.shape, valY.shape))
    return trainX, trainY, valX, valY


def get_full_database(resample, raw_dir, base_dir = "data/"):

    full_data = base_dir + raw_dir + "/"
    data_gen = DataGenerator(random = False, base_dir = full_data)
    data_gen.rewind()
    data_count = (data_gen.steps - 100)
    #data_count = 20000

    final_x = []

    closed_prices = []

    on_new_data = lambda x: final_x.append(x)
    on_closed_price = lambda price: closed_prices.append(price)
    
    print(resample)

    agent = DataAgent(
        resample = resample,
        on_new_data = on_new_data,
        on_closed_price = on_closed_price
    )

    print("Processing {}".format(raw_dir))

    for i in tqdm(range(data_count)):
        agent.on_new_raw_data(data_gen.next())


    closes = pd.DataFrame(closed_prices, columns = ['Close'])

    final_y = get_y_data(closes, -1)
    
    #print(agent.ohlc)
    
    return final_x, final_y

In [4]:
path = "./" 

import data_util
from data_generator import DataGenerator

def load_dataset(dir):
    load_datasets([dir])

def load_datasets(dirs, resample, base_dir = "data/"):
    print(dirs)
    sets = []  
    for raw_dir in dirs:

        x, y = get_full_database(resample = resample,
                                             raw_dir = raw_dir, 
                                             base_dir = base_dir)

        final_data = split(x, y, 0.1, shuffle=False)

        save(final_data, raw_dir)
        sets.append((x, y))
    return sets

def conc_sets(sets):
    trainX = sets[0][0]
    trainY = sets[0][1]
    for i in range(1,  len(sets)):
        data_set = sets[i]
        trainX = np.append(data_set[0], trainX, axis = 0)
        trainY = np.append(data_set[1], trainY, axis = 0)
    
    trainX, trainY, valX, valY = split(trainX, trainY, 0.1, shuffle=True)
    
    return trainX, trainY, valX, valY



In [4]:
%%time

#dirs = ["omgusd", "btceur", "btcusd", "ethusd", "ltcusd", "btcusd17", "btcusdAug19"]
dirs = ["omgusd", "btceur", "btcusd", "ethusd", "ltcusd"]
#dirs = ["omgusd"]

#sets = load_datasets(dirs, '2Min')
#save(conc_sets(sets))

['omgusd', 'btceur', 'btcusd', 'ethusd', 'ltcusd']
2Min
Processing omgusd


<ipython-input-2-37816dab1e21>:55: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(data_count)):


  0%|          | 0/213310 [00:00<?, ?it/s]

Completed: (297, 16) (297,) (33, 16) (33,)
Saving ./data/omgusdtrainX.npy with (297, 16)
2Min
Processing btceur


<ipython-input-2-37816dab1e21>:55: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(data_count)):


...

  0%|          | 0/639540 [00:00<?, ?it/s]

Completed: (999, 16) (999,) (111, 16) (111,)
Saving ./data/ethusdtrainX.npy with (999, 16)
2Min
Processing ltcusd


<ipython-input-2-37816dab1e21>:55: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(data_count)):


  0%|          | 0/543146 [00:00<?, ?it/s]

Completed: (774, 16) (774,) (86, 16) (86,)
Saving ./data/ltcusdtrainX.npy with (774, 16)
Completed: (4127, 16) (4127,) (459, 16) (459,)
Saving ./data/trainX.npy with (4127, 16)
CPU times: user 9min 48s, sys: 4min 50s, total: 14min 38s
Wall time: 20min 11s


In [9]:
def save_database(name):
    x, y = get_full_database('2Min', name, base_dir = "data/")
    final_data = split(x, y, 0.1)
    save(final_data, name)

In [10]:
save_database("backtest")
save_database("4jul21")

2Min
Resample 2Min - TecAn ( windows 20, windows_limit 100 )
Processing backtest


<ipython-input-3-37816dab1e21>:55: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(data_count)):


  0%|          | 0/1585895 [00:00<?, ?it/s]

Completed: (1825, 16) (1825,) (203, 16) (203,)
Saving ./data/backtesttrainX.npy with (1825, 16)
2Min
Resample 2Min - TecAn ( windows 20, windows_limit 100 )
Processing 4jul21


<ipython-input-3-37816dab1e21>:55: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(data_count)):


  0%|          | 0/301794 [00:00<?, ?it/s]

Completed: (270, 16) (270,) (31, 16) (31,)
Saving ./data/4jul21trainX.npy with (270, 16)


In [11]:
x, y = get_full_database('2Min', "4jul21", base_dir = "data/")

2Min
Resample 2Min - TecAn ( windows 20, windows_limit 100 )
Processing 4jul21


<ipython-input-3-37816dab1e21>:55: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(data_count)):


  0%|          | 0/301794 [00:00<?, ?it/s]